In [103]:
import io
from google.colab import files
from keras.models import Sequential
import tensorflow as tf
from keras.layers import Activation, Dense, Dropout, LSTM
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
from math import isnan
import seaborn as sn
import datetime
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from google.colab import drive
%matplotlib inline
import requests
import networkx as nx

In [104]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Inspect Data

In [105]:
with open('/content/drive/My Drive/MachineLearningProject/Dataset/bitstampUSD_1-min_data_2012-01-01_to_2020-12-31.csv', 'r') as f:
  hist = pd.read_csv(f)

In [106]:
hist.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
hist.tail(10)

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
4727767,1609372260,28836.97,28849.67,28807.78,28826.52,2.051917,59173.965272,28838.376450
4727768,1609372320,28826.49,28844.25,28816.09,28816.09,0.721410,20803.457100,28837.207539
4727769,1609372380,28814.36,28822.71,28800.00,28800.00,1.529671,44076.572351,28814.411945
4727770,1609372440,28800.00,28832.79,28800.00,28831.35,2.036450,58675.679644,28812.732950
4727771,1609372500,28809.07,28825.50,28800.01,28810.08,0.087391,2517.799582,28810.597267
4727772,1609372560,28801.47,28829.42,28785.64,28829.42,0.965221,27804.572129,28806.429798
4727773,1609372620,28829.42,28863.90,28829.42,28857.06,2.368831,68332.350629,28846.441863
4727774,1609372680,28850.49,28900.52,28850.49,28882.82,2.466590,71232.784464,28879.056266
4727775,1609372740,28910.54,28911.52,28867.60,28881.30,7.332773,211870.912660,28893.695831
4727776,1609372800,28893.21,28928.49,28893.21,28928.49,5.757679,166449.709320,28909.166061


In [108]:
hist.shape

(4727777, 8)

In [109]:
hist_copy = hist.copy()
hist.shape

(4727777, 8)

In [110]:
hist["Close"].max()

28997.38

In [111]:
hist["Open"].max()

28997.38

In [112]:
hist.drop('Timestamp', axis=1, inplace=True)

In [113]:
hist.isnull().sum()

Open                 1243472
High                 1243472
Low                  1243472
Close                1243472
Volume_(BTC)         1243472
Volume_(Currency)    1243472
Weighted_Price       1243472
dtype: int64

In [114]:
def replace_NaN(df):
  old_df = df.copy()
  for row_index, row in old_df.iterrows():
     column_index = 0
     for value in row:
       if isnan(value):
         df.iloc[row_index, column_index] = df.iloc[row_index-1, column_index] 
       column_index += 1

In [115]:
def replace_NaN_with_median(data):
  data['Open'] = data['Open'].fillna(data['Open'].median())
  data['High'] = data['High'].fillna(data['High'].median())
  data['Low'] = data['Low'].fillna(data['Low'].median())
  data['Close'] = data['Close'].fillna(data['Close'].median())
  data['Volume_(BTC)'] = data['Volume_(BTC)'].fillna(data['Volume_(BTC)'].median())
  data['Volume_(Currency)'] = data['Volume_(Currency)'].fillna(data['Volume_(Currency)'].median())
  data['Weighted_Price'] = data['Weighted_Price'].fillna(data['Weighted_Price'].median())


In [116]:
hist =  hist.dropna(axis = 0, how = 'any')

In [117]:
hist.isnull().sum() #Should be zero

Open                 0
High                 0
Low                  0
Close                0
Volume_(BTC)         0
Volume_(Currency)    0
Weighted_Price       0
dtype: int64

# Train-Test split

In [118]:
def train_test_split(array, test_size=0.2):
    split_row = len(array) - int(test_size * len(array))
    train_data = array[:split_row]
    test_data = array[split_row:]
    return train_data, test_data
    
train, test = train_test_split(hist, test_size=0.2)

In [119]:
target_col = 'Close'
target_index = 3
hist.iloc[1:5, target_index]

478     4.39
547     4.57
548     4.58
1224    4.58
Name: Close, dtype: float64

In [120]:
def line_plot(line1, line2, label1=None, label2=None, title='Train-Test split', lw=2):
    fig, ax = plt.subplots(1, figsize=(16, 7))
    ax.plot(line1, label=label1, linewidth=lw)
    ax.plot(line2, label=label2, linewidth=lw)
    ax.set_ylabel('Close Price', fontsize=14)
    ax.set_title(title, fontsize=16)
    ax.legend(loc='best', fontsize=16)

In [121]:
#line_plot(train[target_col], test[target_col], 'training', 'Train-Test split', title='')

In [122]:
plot_cols = ['Open', 'High', 'Close']
plot_features = hist[plot_cols]


In [123]:
#_ = plot_features.plot(subplots=True)

In [124]:
plot_cols = ['Volume_(BTC)', 'Volume_(Currency)', 'Weighted_Price']
plot_features = hist[plot_cols]


In [125]:
#_ = plot_features.plot(subplots=True)

# Normalization

In [126]:
#scaler = StandardScaler()
#hist_array = scaler.fit_transform(hist)
#hist = pd.DataFrame(hist_array, columns=hist_copy.columns.drop('Timestamp'))

In [127]:
target = hist[target_col].to_numpy()
target = target.reshape(-1,1)
scaler = MinMaxScaler(feature_range = (0,1))

scaler2 = MinMaxScaler(feature_range = (0,1))
scaler2.fit(target)

hist_array = scaler.fit_transform(hist)
hist = pd.DataFrame(hist_array, columns=hist_copy.columns.drop("Timestamp"))

In [128]:
target.shape

(3484305, 1)

In [129]:
hist.tail()

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
3484300,0.993243,0.993763,0.993853,0.994207,0.000165,0.002662,0.993808
3484301,0.994207,0.994952,0.995365,0.995161,0.000405,0.006541,0.995189
3484302,0.994934,0.996215,0.996092,0.996049,0.000421,0.006819,0.996314
3484303,0.997005,0.996594,0.996683,0.995997,0.001253,0.020283,0.996819
3484304,0.996407,0.997179,0.997568,0.997624,0.000984,0.015934,0.997353


In [130]:
hist.tail()

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
3484300,0.993243,0.993763,0.993853,0.994207,0.000165,0.002662,0.993808
3484301,0.994207,0.994952,0.995365,0.995161,0.000405,0.006541,0.995189
3484302,0.994934,0.996215,0.996092,0.996049,0.000421,0.006819,0.996314
3484303,0.997005,0.996594,0.996683,0.995997,0.001253,0.020283,0.996819
3484304,0.996407,0.997179,0.997568,0.997624,0.000984,0.015934,0.997353


In [131]:
hist.shape

(3484305, 7)

# Data Windowing

In [132]:
def extract_window_data(array, window_len=30):
  X = []
  y  = []
  for i in range(window_len, len(array)-window_len):
      X.append(array[i-window_len:i])
      y.append(array[i:i+ int(window_len/3), target_index])
  X, y = np.array(X), np.array(y)
  return X, y

In [133]:
def prepare_data(array, target_col, window_len=30, test_size=0.2):
    #We split the dataset into train and test
    train_data, test_data = train_test_split(array, test_size=test_size)
    #We extract all the possible windows
    X_train, y_train = extract_window_data(train_data, window_len)
    #We extract all the possible windows
    X_test, y_test = extract_window_data(test_data, window_len)
   
    return X_train, y_train, X_test, y_test

# LSTM Model

In [134]:
def build_lstm_model(window_len, num_features, output_size, neurons=50, activ_func='linear', dropout=0.2, loss='mse', optimizer='adam'):
    model = Sequential()
    model.add(LSTM(neurons, input_shape =(window_len, num_features)))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))
    model.compile(loss=loss, optimizer=optimizer, metrics=['mae'])
    
    return model

**Parameters**

In [135]:
np.random.seed(42)
window_len = 30
test_size = 0.2
lstm_neurons = 100
epochs = 2
num_features = len(hist.columns)
activ_func = 'linear'
batch_size = 256
loss = 'mape'
dropout = 0.2
output_size = 10
optimizer = keras.optimizers.Adam(learning_rate=0.00003)

In [136]:
hist_array.shape

(3484305, 7)

In [137]:
X_train, y_train, X_test, y_test = prepare_data(hist_array, target_col, window_len=window_len, test_size=test_size)

In [138]:
X_train.shape

(2787384, 30, 7)

In [139]:
y_train.shape

(2787384, 10)

In [140]:
model = build_lstm_model(window_len, num_features, output_size=output_size, neurons=lstm_neurons, dropout=dropout, loss=loss, optimizer=optimizer)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               43200     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1010      
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 44,210
Trainable params: 44,210
Non-trainable params: 0
_________________________________________________________________


In [141]:
history = model.fit(X_train, y_train, batch_size = batch_size, epochs=epochs, verbose=1, shuffle=True)

Epoch 1/2
10889/10889 [==============================] - 1127s 102ms/step - loss: 23.4989 - mae: 0.0174
Epoch 2/2
10889/10889 [==============================] - 1110s 102ms/step - loss: 14.5876 - mae: 0.0073


In [142]:
history.history

{'loss': [18.479122161865234, 15.206027030944824],
 'mae': [0.010892629623413086, 0.006913251709192991]}

In [ ]:
results = model.evaluate(X_test, y_test)

In [ ]:
preds = model.predict(X_test)

In [ ]:
preds.shape

In [ ]:
predictions = scaler2.inverse_transform(preds)
real_values = scaler2.inverse_transform(y_test)

In [ ]:
def plot(real_values, predictions,  plot_col='Close', max_subplots=5, print_step  = 100000):
  plt.figure(figsize=(12, 8))
  x = []
  for i in range(10):
    x.append(i+1) 
  for n in range(max_subplots):
    plt.subplot(max_subplots, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.scatter(x, real_values[print_step * n], edgecolors='k', label='Labels', c='#2ca02c', s=64)
    plt.scatter(x, predictions[print_step * n], marker='X', edgecolors='k', label='Predictions',c='#ff7f0e', s=64)

  plt.xlabel('Time [10min]')


In [ ]:
# Visualizing the results
plt.plot(real_values, color = 'red', label = 'Real')
plt.plot(predictions, color = 'blue', label = 'Predicted')
plt.title('Bitcoin Close Price min')
plt.ylabel('Close price')
plt.show()

In [ ]:
 plot(real_values, predictions,  plot_col='Close', max_subplots=6, print_step = 100000)

In [ ]:
 plot(real_values, predictions,  plot_col='Close', max_subplots=2, print_step = 690000)

In [ ]:
# Calling `save('my_model.h5')` creates a h5 file `my_model.h5`.
model.save("/content/drive/My Drive/MachineLearningProject/nn_for_bitcoin_price_20min.h5")




# Convolutional model

In [ ]:
np.random.seed(42)
window_len = 30
test_size = 0.05
epochs = 5
num_features = len(hist.columns)
batch_size = 128
loss = 'mape'
dropout = 0.2
output_size = 10
optimizer = keras.optimizers.Adam(learning_rate=0.00001)

In [ ]:
def build_conv_model(output_size, neurons=512, loss='mape', optimizer='adam'):
    CONV_WIDTH = 3
    model = Sequential()
    model.add(tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]))
    model.add(tf.keras.layers.Conv1D(neurons, activation='relu', kernel_size=(CONV_WIDTH)))
    model.add(tf.keras.layers.Dense(output_size, kernel_initializer=tf.initializers.zeros()))
    model.add(tf.keras.layers.Reshape([output_size]))
    model.compile(loss=loss, optimizer=optimizer, metrics=['mape'])
    return model

In [ ]:
model = build_conv_model(output_size=output_size, loss=loss, optimizer=optimizer)

In [ ]:
history = model.fit(X_train, y_train, batch_size = batch_size, epochs=epochs, verbose=1, shuffle=True)

In [ ]:
# Calling `save('my_model.h5')` creates a h5 file `my_model.h5`.
model.save("/content/drive/My Drive/PredictionModel/last_7_days_cnn_for_bitcoin_price_10min.h5")

In [ ]:
results = model.evaluate(X_test, y_test)
print("test loss:", results)

In [ ]:
preds = model.predict(X_test)

In [ ]:
y_test.shape

In [ ]:
predictions = scaler2.inverse_transform(preds)
real_values = scaler2.inverse_transform(y_test)

In [ ]:
predictions.shape

In [ ]:
# Visualizing the results
plt.plot(y_test, color = 'red', label = 'Real')
plt.plot(preds, color = 'blue', label = 'Predicted')
plt.title('Bitcoin Close Price min')
plt.ylabel('Close price')
plt.show()

In [ ]:
# Visualizing the results
plt.plot(real_values[70000], color = 'grey', label = 'Real')
plt.plot(predictions[70000], color = 'blue', label = 'Predicted')
plt.title('Bitcoin Close Price min')
plt.ylabel('Close price')
plt.show()

In [ ]:
# Visualizing the results
plt.plot(real_values[-30], color = 'grey', label = 'Real')
plt.plot(predictions[-30], color = 'blue', label = 'Predicted')
plt.title('Bitcoin Close Price min')
plt.ylabel('Close price')
plt.show()

In [ ]:
 plot(real_values, predictions,  plot_col='Close', max_subplots=6, print_step = 100000)

In [ ]:
 plot(real_values, predictions,  plot_col='Close', max_subplots=2, print_step = 690000)

In [ ]:
 plot(real_values, predictions,  plot_col='Close', max_subplots=5, print_step = 50000)

In [ ]:
 plot(real_values, predictions,  plot_col='Close', max_subplots=3, print_step = 120000)